<h1>Efficient Entity Resolution with Linear Algebra</h1>
<h3>Introduction</h3>
I get a lot of applied work projects that involve the process of linking entity or individual names from different datasets together. Fortunately, there are a lot of different native python packages that provide the ability to do this, from <b>record linkage</b>, <b>dedupe</b>, <b>fuzzywuzzy</b>, <b>splink</b> and more. I'll spend some time exploring these packages (and others) in another post, but I'm mostly interested in looking specifically at a lesser known package, which does not appear to be actively maintained, called <a href="https://github.com/Bergvca/string_grouper"><b>string-grouper</b></a>, and its dependency, which is still being actively maintained, <a href="https://github.com/ing-bank/sparse_dot_topn"><b>sparse-dot-topn</b></a>. 

You can read more about sparse-dot-topn in this Medium article <a href="https://medium.com/wbaa/even-faster-string-comparison-2778be7fe480">here</a>, and check out the source code on PyPI <a href="https://pypi.org/project/sparse-dot-topn/">here</a>. 

In this post, I'm going to take a step back and explore how linear algebra is used to determine whether two strings are similar. I'll delve into the concepts behind this approach and demonstrate why it can be faster than traditional methods used in packages like fuzzywuzzy.

### Term Frequency-Inverse Document Frequency (TF-IDF)
Lets start by taking two vectors: $v1 = [yo, mo]$ and $v2 = [yu, mu]$, we want to run these through some sort of process to determine whether or not the values in both of these matrices are similar. We'll do that first by using a numerical statistic that will tell us how important each character (letter) is in all the strings in our two vectors. 

When using TF-IDF, we'll refer all the strings in both vectors as a <b>corpus</b> and we'll refer to each string as a 'document'. So $yo$, $mo$, $yu$, and $mu$ are all the <b>documents</b> we have in our <b>corpus</b>.

<b>Term Frequency</b> measure how frequently a pre-defined term (letters in our case) occurs in a document. This can literally just be the count of the number of times the letter appears in the document, to other normalization techniques that we'll discuss later. 

<!-- Let's start by taking a look at the document $yo$, which has the letters $y$ and $o$. There are two letters in the string, and each letter occurs once, so the term frequency of each letter would be $1/2$.  -->

Let's start by determining the frequency of $y$ in our corpus, if we're just going to take the raw count of $y$, we can see that it occurs one time, so the term-frequency of $y$ in the document $yo$ is 1. 

$TF(y) = 1$



<!-- There are four total documents, and 2 of them contain the letter $y$, so the term frequency of $y$ is 0.5. The same holds true for $o$ because it occurs once in our document out of two possible characters.  -->


<b>Inverse Document Frequency</b> looks at how important a term (again, a letter in this case) is in our corpus by taking a look at how often it appears across all documents. Terms that appear in many documents will have a lower IDF value. The formula for IDF is: 

$\text{IDF}(t) = \ln\left(\frac{N + 1}{\text{DF}(t) + 1}\right) + 1$

where $t$ is the term, $N$ is the total number of documents in the corpus, and $DF(t)$ is the number of documents that contain the term $t$ (document frequency of $t$). 


Let's calculate the IDF for $y$. There are two documents in the vector $v1$, so $N = 2$. In addition, there is 1 document that contains the letter $y$, so $DF(y) = 1$ Let's go aheada and substitute these values into the formula: 

$\text{IDF}(y) = \ln\left(\frac{2 + 1}{1 + 1}\right) + 1 $


When we finish this calculation we get:

$\text{IDF}(y) = \log\left(\frac{3}{2}\right) + 1 \approx 1.405$

Next we multiple the Term-Frequency (TF) and Inverse Document Frequency (IDF) together: $1 * 1.405 $, which equals $1.405, so we can say that the $TF-IDF$ of the character $y$ in the corpus $v1$ is $1.405$. 

We can run through the same process for the letters $m$ and $o$: 

For $o$, the only difference is the fact that it appears in both strings in $v1$:

$\text{IDF}(o) = \ln\left(\frac{2 + 1}{2 + 1}\right) + 1 = 1$

For $m$, has the same results as $y$:

$\text{IDF}(y) = \ln\left(\frac{2 + 1}{1 + 1}\right) + 1 \approx 1.405$


Now that we know the $TF-IDF$ scores for each letter, we can display them in a table like this: 

<table>
    <tr>
        <th>String</th>
        <th>TF-IDF(m)</th>
        <th>TF-IDF(o)</th>
        <th>TF-IDF(y)</th>
    </tr>
    <tr>
        <td>yo</td>
        <td>0.0</td>
        <td>1.0</td>
        <td>1.405</td>
    </tr>
    <tr>
        <td>mo</td>
        <td>1.405</td>
        <td>1.0</td>
        <td>0.0</td>
    </tr>
</table>

The first row represents the string $yo$ as a vector $[0.0, 1.0, 1.405]$, and the second row represents the string $mo$ as the vector $[1.405, 1.0, 0.0]$.

We have successfully represented the vector $v1$ with numerical values--next we'll add the second vector $v2$ into the mix and see how frequently the terms (letters) in that vector occur in the $v1$ document, which will help us assess the similarity between the documents in both vectors in the final step. 

As a reminder to avoid scrolling up again, $v2 = yu, mu$, we want to calculate the following TF-IDF scores for each letter in $v2$. The important factor to remember here is that we're calculating these $TF-IDF$ scores as if these values existed in $v1$, not in $v2$. So if a letter in $v2$ doesn't exist in $v1$, it'll have a $TF-IDF$ score of 0. 

Both $y$ and $m$ exist in $v1$ so their values would be the same as we calculated above, but $u$ is not in $v1$, so its $TF-IDF$ score will be 0. 

We can display the resulting table for $v2$ as follows:

<table>
    <tr>
        <th>String</th>
        <th>TF-IDF(m)</th>
        <th>TF-IDF(u)</th>
        <th>TF-IDF(y)</th>
    </tr>
    <tr>
        <td>yu</td>
        <td>0.0</td>
        <td>0.0</td>
        <td>1.405</td>
    </tr>
    <tr>
        <td>mu</td>
        <td>1.405</td>
        <td>0.0</td>
        <td>0.0</td>
    </tr>
</table>

So we've successfully been able to convert both our vectors to TF-IDF matrices (displayed above as tables). Now we can actually determine whether the strings between both vectors are similar or not. 

In order to do this, we'll have to use some <b>linear algebra</b>, more specifically, <b>cosine similarity</b>.

### Cosine Similarity Between Two Vectors
So now we have our two vectors represented as matrices like this: 

$v1 = \begin{bmatrix}
0 & 1 & 1.40546511 \\
1.40546511 & 1 & 0
\end{bmatrix}$

$v2 = \begin{bmatrix}
0 & 0 & 1.40546511 \\
1.40546511 & 0 & 0
\end{bmatrix}$

Remember that each row in each vector represents one of the strings and each column represents a letter from the strings in the vector (each vector has three letters represented in two vectors, hence the 2 rows and 3 columns). 

In order to calculate the cosine similarity, we'll go ahead and use the formula:

$\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$

We need to calculate the cosine similarity between the rows in each vector against the rows in the other. So we'll need to find the cosine similarity between the first row in $v1$ against the first row in $v2$... etc. In the end, we'll have a matrix like the table below, except we'll have solutions to each of these cosine similarities. Remember that each row represents one of the strings in the original vector--and each row is a TF-IDF representation of that string:

<table>
    <tr>
        <th></th>
        <th>v2 Row 1 (yu)</th>
        <th>v2 Row 2 (mu)</th>
    </tr>
    <tr>
        <th>v1 Row 1 (yo)</th>
        <td>cosine_similarity(v1 Row 1, v2 Row 1)</td>
        <td>cosine_similarity(v1 Row 1, v2 Row 2)</td>
    </tr>
    <tr>
        <th>v1 Row 2 (mo)</th>
        <td>cosine_similarity(v1 Row 2, v2 Row 1)</td>
        <td>cosine_similarity(v1 Row 2, v2 Row 2)</td>
    </tr>
</table>

Let's calculate the cosine similarity of the first row of $v2$, which represents the string 'yu', and the first row of $v1$, which represents the string 'yo'. 


### Calculating TF-IDF and Cosing Similarity of Matrices Using Scikit-Learn
So we represented each string in our two vectors as TF-IDF matrices, and then calculated their cosine similarities to determine whether or not they were actually similar or not. We could write this all up in python, however, there are fortunately python packages that do all of this work for us. We're going to use scikit-learn's TfidfVectorizer() and cosine_similarity() methods in order to do it. 

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Let's start by first defining our two vectors, $v1$ and $v2$ with the same strings as we did above. 

In [2]:
v1 = ['yo', 'mo']
v2 = ['yu', 'mu']

Next, let's create our TfidfVectorizer object, where we'll set the analyzer to 'char', so that it treats every letter as a "term" (commonly referred to as a "token" in this instance), and break it up into ngrams that are 1-character long as defined in the ngram_range. Lastly, we'll set the normalization to "None" for now so that we can get the same results we got above. 

In [3]:
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 1), norm=None)

Now we can go ahead and use the method fit() so that our TfidfVectorizer can learn the vocabulary from our $v1$ vector.

In [4]:
vectorizer.fit(v1)

TfidfVectorizer(analyzer='char', norm=None)

Finally, we can use the transform() method to transform the $v1$ vector to a TF-IDF matrix. I'll assign the results to the variable X. 

In [5]:
X1 = vectorizer.transform(v1)

You can also streamline this by using the method fit_transform(), which combines fit() and transform() into one step:

In [6]:
X1 = vectorizer.fit_transform(v1)

We can take a look at the results of our $v1$ matrix, notice that it looks the same as our matrix from above. The first row represents the vector $yo$, and the second represents $mo$, while the first, second, and third columns represent the characters $m, o$ and $y$ respectively. 

In [7]:
X1.toarray()

array([[0.        , 1.        , 1.40546511],
       [1.40546511, 1.        , 0.        ]])

Our TfidfVectorizer object has already been fitted to $v1$, so all we need to do is transform() the $v2$ vector against it, and it'll do all the work we did above for completing our TF-IDF vector arithmetic. 

In [8]:
X2 = vectorizer.transform(v2)

We can take a look at the array, and it's identical to the one we calculated before--the first and second rows represent $yu$ and $mu$ respectively, while the first, second, and third columns represent the characters $m$, $u$, and $y$ respectively. 

In [9]:
X2.toarray()

array([[0.        , 0.        , 1.40546511],
       [1.40546511, 0.        , 0.        ]])

Lastly, we can use scikit-learn's built in method to calculate the cosine similarity between these two matrices.

In [10]:
cosine_similarity(X1, X2)

array([[0.81480247, 0.        ],
       [0.        , 0.81480247]])

As we did above, the rows in the matrix above represent $yu$ and $mu$ while the columns represent $yo$ and $mo$. We can see that the cosine similarity between $yo$ and $yu$ is about 0.82, while the cosine similarity between $mo$ and $mu$ is the same. The comparisons between $yo$ and $mo$ and $yo$ and $mu$ are 0. 